In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from Master import Master
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from LightXBM import LightXBM
from LightGMB import LightGMB

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [2]:
target_A = pd.read_parquet("A/train_targets.parquet").assign(time=lambda df: pd.to_datetime(df['time']))
#test_A=pd.read_parquet("A/X_test_estimated.parquet")
A_observed=pd.read_parquet("A/X_train_observed.parquet")
A_estimated=pd.read_parquet("A/X_train_estimated.parquet").drop(columns=['date_calc'])
target_B = pd.read_parquet("B/train_targets.parquet").assign(time=lambda df: pd.to_datetime(df['time']))
B_observed = pd.read_parquet("B/X_train_observed.parquet")
B_estimated=pd.read_parquet("B/X_train_estimated.parquet").drop(columns=['date_calc'])
target_C = pd.read_parquet("C/train_targets.parquet").assign(time=lambda df: pd.to_datetime(df['time']))
C_observed=pd.read_parquet("C/X_train_observed.parquet")
C_estimated=pd.read_parquet("C/X_train_estimated.parquet").drop(columns=['date_calc'])

# test sets
test_A=pd.read_parquet("A/X_test_estimated.parquet").drop(columns=['date_calc'])
test_B=pd.read_parquet("B/X_test_estimated.parquet").drop(columns=['date_calc'])
test_C=pd.read_parquet("C/X_test_estimated.parquet").drop(columns=['date_calc'])


In [3]:
test_A.head()

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2023-05-01 00:00:00,4.4,1.286,912.700012,0.0,0.0,1041.199951,0.0,271.700012,0.0,0.0,0.0,0.0,80.699997,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.900024,0.0,0.0,1000.700012,1007.000000,0.0,0.0,80.300003,1013.299988,NaN,0.0,0.0,0.0,0.0,10.520000,-11.185,0.0,273.799988,80.699997,30210.699219,4.0,2.2,3.4,-0.0
1,2023-05-01 00:15:00,4.3,1.287,912.700012,0.0,0.0,1054.800049,0.0,271.700012,0.0,0.0,0.0,0.0,77.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.700012,0.0,0.0,1000.599976,1006.900024,0.0,0.0,80.300003,1013.200012,NaN,0.0,0.0,0.0,0.0,14.203000,-10.825,0.0,273.799988,77.000000,30003.599609,4.0,2.1,3.4,-0.0
2,2023-05-01 00:30:00,4.3,1.287,912.700012,0.0,0.0,1068.300049,0.0,271.600006,0.0,0.0,0.0,0.0,73.099998,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.599976,0.0,0.0,1000.500000,1006.700012,0.0,0.0,80.300003,1013.000000,NaN,0.0,0.0,0.0,0.0,17.868999,-10.360,0.0,273.799988,73.099998,29797.099609,3.9,2.1,3.3,-0.0
3,2023-05-01 00:45:00,4.3,1.287,912.700012,0.0,0.0,1081.900024,0.0,271.600006,0.0,0.0,0.0,0.0,69.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.500000,0.0,0.0,1000.400024,1006.599976,0.0,0.0,80.199997,1012.900024,NaN,0.0,0.0,0.0,0.0,21.514000,-9.794,0.0,273.799988,69.000000,29618.599609,3.9,2.0,3.3,-0.0
4,2023-05-01 01:00:00,4.3,1.287,NaN,0.0,0.0,1095.400024,0.0,271.600006,0.0,0.0,0.0,0.0,64.500000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1013.299988,0.0,0.0,1000.200012,1006.500000,0.0,0.0,80.199997,1012.799988,NaN,0.0,0.0,0.0,0.0,25.135000,-9.128,0.0,273.799988,64.500000,29507.500000,3.9,2.0,3.3,-0.0


In [4]:
A_estimated.head(50)

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2022-10-28 22:00:00,8.4,1.233,1425.099976,0.000000,0.000000,1211.699951,1.0,281.399994,0.000000,0.000000,0.000000,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1001.099976,0.00,0.0,988.500000,994.500000,0.0,0.0,92.699997,1000.599976,NaN,0.0,0.0,-0.0,0.4,340.799011,-38.466999,0.0,284.700012,100.000000,29429.699219,0.8,-0.4,0.7,-0.0
1,2022-10-28 22:15:00,8.4,1.233,2085.899902,0.000000,0.000000,1560.500000,1.0,281.299988,0.000000,0.000000,0.000000,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1000.799988,0.00,0.0,988.200012,994.200012,0.0,0.1,92.699997,1000.299988,NaN,0.0,0.0,-0.0,0.4,345.451996,-38.955002,0.0,284.700012,100.000000,23604.099609,0.7,-0.4,0.7,-0.0
2,2022-10-28 22:30:00,8.3,1.233,2746.600098,0.000000,0.000000,1909.400024,1.0,281.200012,0.000000,0.000000,0.000000,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1000.400024,0.00,0.0,987.900024,993.900024,0.0,0.1,92.699997,1000.000000,NaN,0.0,0.0,-0.0,0.5,350.153015,-39.310001,0.0,284.700012,100.000000,17799.800781,0.7,-0.3,0.6,-0.0
3,2022-10-28 22:45:00,8.3,1.233,3407.399902,0.000000,0.000000,2258.199951,1.0,281.200012,0.000000,0.000000,0.000000,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1000.099976,0.00,0.0,987.599976,993.599976,0.0,0.1,92.599998,999.700012,NaN,0.0,0.0,-0.0,0.6,354.886993,-39.528999,0.0,284.600006,100.000000,12016.500000,0.6,-0.2,0.6,-0.0
4,2022-10-28 23:00:00,8.2,1.233,4068.199951,0.000000,0.000000,2607.000000,1.0,281.100006,0.000000,0.000000,0.000000,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,999.799988,0.00,0.0,987.299988,993.299988,0.0,0.1,92.599998,999.299988,NaN,0.0,0.0,-0.0,0.6,359.638000,-39.609001,0.0,284.600006,100.000000,6254.399902,0.6,-0.1,0.6,-0.0
5,2022-10-28 23:15:00,8.1,1.234,4046.199951,0.000000,0.000000,2242.300049,1.0,280.899994,0.000000,0.000000,0.000000,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,999.599976,0.00,0.0,987.099976,993.099976,0.0,0.1,92.800003,999.200012,NaN,0.0,0.0,-0.0,0.7,4.391000,-39.549999,0.0,284.399994,100.000000,5836.399902,0.7,0.2,0.7,-0.0
6,2022-10-28 23:30:00,8.1,1.234,4024.199951,0.000000,0.000000,1877.500000,1.0,280.799988,0.000000,0.000000,0.000000,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,999.400024,0.00,0.0,986.900024,992.900024,0.0,0.1,93.099998,999.000000,NaN,0.0,0.0,-0.0,0.7,9.129000,-39.352001,0.0,284.200012,100.000000,5415.200195,0.8,0.7,0.5,-0.0
7,2022-10-28 23:45:00,8.0,1.235,4002.199951,0.000000,0.000000,1512.699951,1.0,280.600006,0.000000,0.000000,0.000000,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,999.299988,0.00,0.0,986.700012,992.700012,0.0,0.1,93.300003,998.799988,NaN,0.0,0.0,-0.0,0.8,13.835000,-39.018002,0.0,284.100006,100.000000,4990.700195,1.0,0.9,0.3,-0.0
8,2022-10-29 00:00:00,8.2,1.230,1978.000000,0.000000,0.000000,1660.500000,1.0,281.000000,0.000000,0.000000,0.000000,0.000000,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,997.200012,0.08,1.0,984.700012,990.700012,0.0,0.2,89.199997,996.700012,NaN,0.0,0.0,-0.0,1.4,17.361000,-39.013000,0.1,283.700012,100.000000,3041.600098,1.5,1.2,0.8,-0.0
9,2022-10-29 00:15:00,8.2,1.230,1914.000000,0.000000,0.000000,1670.199951,1.0,280.899994,0.000000,0.000000,0.000000,0.000000,100.000000,6.0,0.0,0.0,

In [5]:
time_diffs = test_A["date_forecast"].diff().value_counts()
print(time_diffs)

0 days 00:15:00    2862
1 days 00:15:00      10
3 days 00:15:00       3
2 days 00:15:00       2
4 days 00:15:00       1
7 days 00:15:00       1
Name: date_forecast, dtype: int64


In [6]:
C_estimated.tail()

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
17571,2023-04-30 22:45:00,4.4,1.280,1034.599976,0.0,0.0,562.299988,0.0,272.000000,0.0,0.0,0.0,0.0,99.800003,24.0,0.0,0.0,0.4,0.0,0.0,0.0,1.0,1014.400024,0.0,0.0,998.599976,1004.900024,0.0,0.0,80.000000,1011.099976,NaN,0.0,0.0,-0.0,0.1,352.915985,-11.769,0.2,274.200012,99.900002,10681.200195,3.0,2.1,2.2,-0.0
17572,2023-04-30 23:00:00,4.4,1.280,897.200012,0.0,0.0,569.500000,0.0,272.000000,0.0,0.0,0.0,0.0,100.000000,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1014.400024,0.0,0.0,998.500000,1004.799988,0.0,0.0,80.400002,1011.099976,NaN,0.0,0.0,-0.0,0.1,356.630005,-11.923,0.2,274.100006,100.000000,9890.900391,2.8,1.9,2.1,-0.0
17573,2023-04-30 23:15:00,4.4,1.280,1288.699951,0.0,0.0,562.299988,0.0,272.000000,0.0,0.0,0.0,0.0,99.699997,24.0,0.0,0.0,0.4,0.0,0.0,0.0,1.0,1014.200012,0.0,0.0,998.400024,1004.700012,0.0,0.0,80.099998,1010.900024,NaN,0.0,0.0,-0.0,0.1,0.349000,-11.967,0.1,274.200012,100.000000,10716.799805,2.9,1.9,2.2,-0.0
17574,2023-04-30 23:30:00,4.4,1.280,1680.300049,0.0,0.0,555.099976,0.0,271.899994,0.0,0.0,0.0,0.0,99.000000,24.0,0.0,0.0,0.4,0.0,0.0,0.0,1.0,1014.099976,0.0,0.0,998.299988,1004.500000,0.0,0.0,79.800003,1010.799988,NaN,0.0,0.0,-0.0,0.1,4.067000,-11.902,0.1,274.200012,99.800003,11538.799805,3.0,1.9,2.3,-0.0
17575,2023-04-30 23:45:00,4.4,1.279,2071.800049,0.0,0.0,547.799988,0.0,271.899994,0.0,0.0,0.0,0.0,97.699997,24.0,0.0,0.0,0.4,0.0,0.0,0.0,1.0,1013.900024,0.0,0.0,998.099976,1004.400024,0.0,0.0,79.500000,1010.599976,NaN,0.0,0.0,-0.0,0.0,7.780000,-11.728,0.1,274.200012,99.699997,12357.099609,3.0,2.0,2.3,-0.0


Sample rate 15M --> 1T for observed data

In [7]:
A_observed = Master.averaged(A_observed,"date_forecast")
A_estimated = Master.averaged(A_estimated,"date_forecast")
B_observed = Master.averaged(B_observed,"date_forecast")
B_estimated = Master.averaged(B_estimated,"date_forecast")
C_observed = Master.averaged(C_observed,"date_forecast")
C_estimated = Master.averaged(C_estimated,"date_forecast")

#Do so for test data as well
test_A=Master.averaged_test(test_A,"date_forecast")
test_B=Master.averaged_test(test_B,"date_forecast")
test_C=Master.averaged_test(test_C,"date_forecast")

#C_estimated.head()
test_C.head()


/Users/henningdropping/Desktop/FGH23/Maskinlæring/Kaggle/Master.py:72: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_resampled = df_averaged.groupby(['date', 'hour']).mean().reset_index()
/Users/henningdropping/Desktop/FGH23/Maskinlæring/Kaggle/Master.py:72: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_resampled = df_averaged.groupby(['date', 'hour']).mean().reset_index()
/Users/henningdropping/Desktop/FGH23/Maskinlæring/Kaggle/Master.py:72: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify 

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,date_forecast
0,4.15,1.28600,NaN,0.000000,0.000000,1236.500000,0.0,271.024994,0.000000,0.000000,0.00,0.000000,69.300003,24.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1013.775024,0.0,0.0,998.025024,1004.275024,0.0,0.0,80.949997,1010.575012,NaN,0.0,0.0,0.0,0.0,16.025501,-10.57800,0.0,273.700012,69.300003,31241.974609,3.425,1.450,3.100,0.0,2023-05-01 00:00:00
1,4.05,1.28550,1477.099976,0.000000,0.000000,1220.425049,0.0,270.700012,0.000000,0.000000,0.00,0.000000,78.525002,24.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1013.250000,0.0,0.0,997.500000,1003.775024,0.0,0.0,79.949997,1010.075012,NaN,0.0,0.0,0.0,0.0,30.496750,-7.92825,0.0,273.750000,78.525002,31019.550781,3.325,1.400,3.025,0.0,2023-05-01 01:00:00
2,3.90,1.28375,1477.099976,0.000000,0.000000,1258.650024,0.0,270.200012,0.000000,0.000000,0.00,0.000000,89.275002,24.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1012.775024,0.0,0.0,996.950012,1003.224976,0.0,0.0,77.350006,1009.525024,NaN,0.0,0.0,0.0,0.0,44.516251,-3.84375,0.0,273.849976,89.275002,32372.800781,3.225,1.225,3.000,0.0,2023-05-01 02:00:00
3,3.75,1.28275,NaN,9972.500000,11.549999,1655.175049,0.0,269.700012,9.300000,16722.224609,0.00,0.000000,57.750000,24.0,0.0,0.0,0.0,0.0,0.0,0.75,1.0,1012.224976,0.0,0.0,996.375000,1002.650024,0.0,0.0,74.699997,1008.924988,NaN,0.0,0.0,0.0,0.0,58.079498,1.39100,0.0,273.950012,57.750000,35433.625000,3.125,0.950,3.000,0.0,2023-05-01 03:00:00
4,3.70,1.28150,1990.000000,141106.671875,76.675003,1639.099976,0.0,269.450012,47.924999,103007.421875,28.25,50847.101562,60.474998,24.0,0.0,0.0,0.0,0.0,0.0,1.00,0.0,1011.750000,0.0,0.0,995.950012,1002.150024,0.0,0.0,73.575005,1008.450012,NaN,0.0,0.0,0.0,0.0,71.333252,7.45475,0.0,273.950012,60.474998,35455.574219,3.025,0.700,2.950,0.0,2023-05-01 04:00:00


In [8]:
#A_observed_estimated = Master.target_join(A_observed,A_estimated,target_A)
#B_observed_estimated = Master.target_join(B_observed,B_estimated,target_B)
#C_observed_estimated = Master.target_join(C_observed,C_estimated,target_C)
A_observed_estimated  = pd.concat([A_observed,A_estimated],ignore_index=True)
B_observed_estimated  = pd.concat([B_observed,B_estimated],ignore_index=True)
C_observed_estimated  = pd.concat([C_observed,C_estimated],ignore_index=True)
A_observed_estimated, target_A = Master.inner_join(A_observed_estimated,target_A)
B_observed_estimated, target_B = Master.inner_join(B_observed_estimated,target_B)
C_observed_estimated, target_C = Master.inner_join(C_observed_estimated,target_C)


#For just estimated

#A_estimated_only, target_A = Master.inner_join(A_estimated,target_A)
#B_estimated_only, target_B = Master.inner_join(B_estimated,target_B)
#C_estimated_only, target_C = Master.inner_join(C_estimated,target_C)

#For just Observed. 

#A_observed_only, target_A = Master.inner_join(A_observed,target_A)
#B_observed_only, target_B = Master.inner_join(B_observed,target_B)
#C_observed_only, target_C = Master.inner_join(C_observed,target_C)

#A_B_C_est_only=pd.concat([A_estimated_only,B_estimated_only,C_estimated_only])
#A_B_C_obs_only=pd.concat([A_observed_only,B_observed_only,C_observed_only])
A_B_C_all=pd.concat([A_observed_estimated,B_observed_estimated,C_observed_estimated])

#A_B_C_est_only.shape





/Users/henningdropping/Desktop/FGH23/Maskinlæring/Kaggle/Master.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_df.drop(target_df.index, inplace=True)
/Users/henningdropping/Desktop/FGH23/Maskinlæring/Kaggle/Master.py:164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_df.drop(target_df.index, inplace=True)


Dt64 feature --> Two features: days_since_jan, hour

In [9]:
#A_observed_estimated=Master.timestamp(A_observed_estimated,"date_forecast")
#B_observed_estimated=Master.timestamp(B_observed_estimated,"date_forecast")
#C_observed_estimated=Master.timestamp(C_observed_estimated,"date_forecast")

#Uncomment everything above for all data

#A_estimated_only=Master.timestamp(A_estimated_only,"date_forecast")
#B_estimated_only=Master.timestamp(B_estimated_only,"date_forecast")
#B_estimated_only=Master.timestamp(C_estimated_only,"date_forecast")

#A_B_C_est_only=Master.timestamp(A_B_C_est_only,"date_forecast")
#A_B_C_obs_only=Master.timestamp(A_B_C_obs_only,"date_forecast")
A_B_C_all=Master.timestamp(A_B_C_all,"date_forecast")




# Do so for test set also

test_A=Master.timestamp(test_A,"date_forecast")
test_B=Master.timestamp(test_B,"date_forecast")
test_C=Master.timestamp(test_C,"date_forecast")










Concatinating A,B,C with all accessible values, this is jalla jalla

In [10]:
#A_B_C_obs_est = pd.concat([A_observed_estimated,B_observed_estimated,C_observed_estimated],ignore_index=True)
target_A_B_C = pd.concat([target_A,target_B,target_C],ignore_index=True).drop(columns=["time"])


test_A_B_C = pd.concat([test_A,test_B,test_C],ignore_index=True)

#uncomment all above for all data

#A_B_C_est_only = pd.concat([A_estimated_only,B_estimated_only,C_estimated_only],ignore_index=True).drop(columns=["date_forecast"])


#A_B_C_est_only.shape
#target_A_B_C.shape
#test_A_B_C.head()




In [11]:
A_B_C_all.head(50)

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,days_since_jan,hour
0,7.700000,1.22825,1728.949951,0.000000e+00,0.000000,1728.949951,0.00,280.299988,0.000000,0.000000,0.000000,0.000000e+00,99.074997,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1006.299988,0.0000,0.00,993.750000,999.775024,0.0,0.000,71.674995,1005.799988,NaN,0.0,0.0,0.0,0.175,348.036743,-3.774250,0.000,286.225006,100.000000,40386.476562,3.600,-3.575,-0.500,0.0,153,22
1,7.700000,1.22350,1689.824951,0.000000e+00,0.000000,1689.824951,0.00,280.299988,0.000000,0.000000,0.000000,0.000000e+00,99.750000,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1005.200012,0.0000,0.00,992.674988,998.650024,0.0,0.025,68.000000,1004.650024,NaN,0.0,0.0,0.0,0.200,91.980751,-4.357250,0.000,286.899994,100.000000,33770.648438,3.350,-3.350,0.275,0.0,153,23
2,7.875000,1.21975,1563.224976,0.000000e+00,0.000000,1563.224976,0.00,280.649994,0.000000,0.000000,0.000000,0.000000e+00,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.00,1.00,1004.525024,0.0000,0.00,992.000000,997.974976,0.0,0.100,67.949997,1003.950012,NaN,0.0,0.0,0.0,0.400,14.934750,-3.309500,0.000,286.950012,100.000000,13595.500000,3.050,-2.950,0.750,0.0,154,0
3,8.425000,1.21800,1283.425049,2.086500e+02,0.750000,1283.425049,0.00,281.674988,0.300000,526.775024,0.000000,0.000000e+00,100.000000,6.0,0.0,0.0,0.0,0.0,0.0,0.25,1.00,1004.025024,0.0000,0.00,991.500000,997.449951,0.0,0.125,73.875000,1003.449951,NaN,0.0,0.0,0.0,0.550,28.630251,-0.822500,0.000,286.750000,100.000000,2321.850098,2.725,-2.600,0.875,0.0,154,1
4,8.950000,1.21800,1003.500000,3.246815e+04,23.100000,1003.500000,0.00,282.500000,11.975000,22068.949219,0.150000,2.829750e+02,84.875000,6.0,0.0,0.0,0.0,0.0,0.0,1.00,0.00,1003.099976,0.0000,0.00,990.550049,996.500000,0.0,0.100,79.925003,1002.500000,NaN,0.0,0.0,0.0,0.250,41.997501,3.051250,0.000,286.450012,99.224998,11634.799805,2.550,-2.350,0.925,0.0,154,2
5,9.250000,1.21650,809.375000,1.794991e+05,84.375000,809.375000,0.00,283.049988,45.125000,102788.046875,6.300000,1.162755e+04,65.025002,6.0,0.0,0.0,0.0,0.0,0.0,1.00,0.00,1002.349976,0.0000,0.00,989.750000,995.700012,0.0,0.000,82.849998,1001.674988,NaN,0.0,0.0,0.0,0.100,55.041748,8.071000,0.000,286.475006,94.800003,29848.199219,2.300,-2.200,0.800,0.0,154,3
6,9.525000,1.21300,757.775024,4.781178e+05,186.649994,757.775024,0.00,283.524994,89.525002,242438.000000,43.724998,9.007338e+04,77.550003,6.0,0.0,0.0,0.0,0.0,0.0,1.00,0.00,1001.825012,0.0000,0.00,989.250000,995.200012,0.0,0.000,82.275002,1001.174988,NaN,0.0,0.0,0.0,0.000,67.898003,13.956500,0.000,286.825012,95.099998,35980.148438,2.325,-2.200,0.750,0.0,154,4
7,9.700000,1.20750,705.650024,8.926679e+05,311.525024,705.650024,0.00,283.799988,139.000000,411381.906250,53.099998,1.742545e+05,85.050003,6.0,0.0,0.0,0.0,0.0,0.0,1.00,0.00,1001.200012,0.0000,0.00,988.700012,994.599976,0.0,0.000,78.699997,1000.500000,NaN,0.0,0.0,0.0,0.000,80.821251,20.406250,0.000,287.325012,97.150002,37874.523438,2.225,-2.000,0.950,0.0,154,5
8,9.550000,1.20500,669.650024,1.357902e+06,442.750000,669.650024,0.00,283.600006,167.100006,550960.562500,33.825001,1.564732e+05,96.750000,6.0,0.0,0.0,0.0,0.0,0.0,1.00,0.00,1000.750000,0.0000,0.00,988.275024,994.200012,0.0,0.050,75.099998,1000.150024,NaN,0.0,0.0,0.0,0.125,94.178497,27.096001,0.000,287.450012,99.19999

In [12]:

#X_train=A_B_C_est_only
#X_train=A_B_C_obs_only
X_train=A_B_C_all
Y_train = target_A_B_C

to_be_predicted = test_A_B_C

#Need to do some fillin

X_train["snow_density:kgm3"].fillna(0, inplace=True)
X_train = X_train.interpolate().fillna(0)

to_be_predicted["snow_density:kgm3"].fillna(0, inplace=True)
to_be_predicted = to_be_predicted.interpolate().fillna(0)

#to_be_predicted.fillna(0, inplace=True)

null_counts = Y_train.isnull().sum()
print(null_counts)

# Assuming X and y are your data and labels, respectively
model = LightGMB()
model.train(X_train, Y_train)

# Assuming unlabeled_df is your DataFrame with unlabeled data
predictions = model.predict(to_be_predicted)

predictions_df = pd.DataFrame(predictions, columns=['predictions'])  # adjust column name as needed
#predictions_df[predictions_df < 0] = 0
predictions_df['id'] = range(len(predictions_df))
predictions_df.set_index('id', inplace=True)
predictions_df.to_csv('predictions.csv', index=True)



pv_measurement    0
dtype: int64
Fitting 5 folds for each of 50 candidates, totalling 250 fits
[LightGBM] [Warning] min_data_in_leaf is set=56, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=56
[LightGBM] [Warning] min_data_in_leaf is set=56, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=56
[LightGBM] [Warning] min_data_in_leaf is set=56, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=56
[LightGBM] [Warning] min_data_in_leaf is set=56, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=56
[LightGBM] [Warning] min_data_in_leaf is set=56, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=56
[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] min_data_in_leaf is set=14, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=14
[LightGBM] [Warning] min_data_in_leaf is set=14, mi

/Users/henningdropping/anaconda3/lib/python3.11/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[9]	training's l2: 287326
[10]	training's l2: 266711
[11]	training's l2: 248068
[12]	training's l2: 231604
[13]	training's l2: 223792
[14]	training's l2: 209292
[15]	training's l2: 202947
[16]	training's l2: 197227
[17]	training's l2: 192047
[18]	training's l2: 180568
[19]	training's l2: 176331
[20]	training's l2: 172427
[21]	training's l2: 162950
[22]	training's l2: 154283
[23]	training's l2: 151404
[24]	training's l2: 143877
[25]	training's l2: 137098
[26]	training's l2: 130938
[27]	training's l2: 125366
[28]	training's l2: 123552
[29]	training's l2: 118634
[30]	training's l2: 114309
[31]	training's l2: 110189
[32]	training's l2: 106439
[33]	training's l2: 103100
[34]	training's l2: 99987.4
[35]	training's l2: 97233.4
[36]	training's l2: 96248.3
[37]	training's l2: 93750.5
[38]	training's l2: 91482.6
[39]	training's l2: 89410
[40]	training's l2: 87534.5
[41]	training's l2: 85737.4
[42]	training's l2: 84101.9
[43]	training's l2: 82636.1
[44]	training's l2: 82057.4
[45]	training's l2: 

Now for the grand finale of trying it on test data, jalla jalla for now


X_train = A_B_C_obs_est
Y_train = target_A_B_C

to_be_predicted = test_A_B_C

X_train.fillna(0, inplace=True)
to_be_predicted.fillna(0, inplace=True)

null_counts = Y_train.isnull().sum()
print(null_counts)

base_models = [('random_forest', RandomForestRegressor(n_estimators=100, random_state=42))]

# Use XGBoost as the meta-model
#meta_model = XGBRegressor(n_estimators=100, random_state=42)

# Create and fit the stacking regressor
#stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=meta_model)
#stacking_regressor.fit(X_train, Y_train)





model = RandomForestRegressor(n_estimators=1000, random_state=42)  # Create a Random Forest Regressor model
model.fit(X_train, Y_train)  # Train the model with the training data
predictions = model.predict(to_be_predicted)
#predictions = stacking_regressor.predict(to_be_predicted)
predictions_df = pd.DataFrame(predictions, columns=['predictions'])  # adjust column name as needed
predictions_df['id'] = range(len(predictions_df))
predictions_df.set_index('id', inplace=True)
predictions_df.to_csv('predictions.csv', index=True)


